In [1]:
# handle sheet file
    # for each row of sheets
        # handle row data
            # if row is empty
                # skip this row
            # if col_B == "P"
                # handle as parent task
                # if col_A is not empty (parent task is created)
                    # handle update parent task if any changes (compare hash)
                # else
                    # handle as new parent task
            # else
                # handle as task with timesheet
                # if col_A is not empty (timesheet is created)
                    # handle update timesheet if any changes (compare hash)
                # else
                    # handle as new timesheet
    # endfor
    # handle update col_A data of sheets
        # for each row of col_A array above
            # update batch of col_A in sheet
    # end of function

In [22]:
import os
from dateutil import parser
from frappeclient import FrappeClient

In [3]:
conn = FrappeClient("https://acerp.pandion.vn")
conn.login(os.environ['ACERP_ADMINISTRATOR'], os.environ['ACERP_ADMINISTRATOR_PASS'])

{'message': 'Logged In', 'home_page': '/app', 'full_name': 'Administrator'}

In [4]:
from util.google_sheet import GGSheet, mapping_cell_with_dates_raw

In [5]:
EXCEL_TASK_PRIORITY = {
    "": "Medium",
    "1_Urgen": "Urgent",
    "2_Important": "High",
    "3_Medium": "Medium",
    "4_Low": "Low",
}
EXCEL_TYPE_PARENT_TASK = {
    "": "Other",
    "0_Pre CO": "Pre Concept",
    "1_CO": "Concept",
    "2_BD": "Basic Design",
    "3_DD": "Design Development",
    "4_TD": "Tender Doc",
    "5_CD": "Construction",
    "6_AU": "Authorship",
    "7_Other": "Other",
}
EXCEL_TASK_STATUS = {
    "": "Open",
    "1_Open": "Open",
    "2_In progress": "In Progress",
    "3_Pending": "Pending",
    "4_Cancel": "Cancel",
    "5_Done": "Done",
    "6_Review": "Review",
}
EXCEL_TIME_SHEET_STATUS = {
    "": "Draft",
    "Open": "Draft",
    "In Progress": "Draft",
    "Pending": "Draft",
    "Review": "Draft",
    "Done": "Submitted",
    "Cancel": "Cancelled",
}
EXCEL_TIME_SHEET_DOC_STATUS = {
    "": 0,
    "Draft": 0,
    "Submitted": 1,
    "Cancelled": 2,
}

In [6]:
row_of_date=3
row_start=5
row_end=1200
company="ACONS"

In [6]:
url_file="https://docs.google.com/spreadsheets/d/12bDZbCnJVkFTJTeMBmgsrAEejVKQCXa11M951ylOQVw/edit#gid=1885142845"
worksheet_name="BAP"

In [7]:
url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
worksheet_name="T2"

In [8]:
ggSheet = GGSheet(url_file, worksheet_name)

In [9]:
async def get_row_date_header():
    date_values = await ggSheet.get_values_with_excel_style(num_of_row=row_of_date, seed=1)
    ignore_values = ["A", "B", "C", "D", "E", "F", "G", "H", "I", "J", "K", "L", "M", "N", "O", "P"]
    row_date = {k: v for k, v in date_values.items() if k not in ignore_values}
    return row_date

In [10]:
def mapping_cell_with_attr(row_data):
    attrs = {}
    attrs["import_key"]          = row_data["A"] if "A" in row_data else ""
    attrs["type"]                = row_data["B"] if "B" in row_data else ""
    attrs["project_code"]        = row_data["C"] if "C" in row_data else ""
    attrs["task"]                = row_data["O"] if "O" in row_data else ""
    attrs["activity_code"]       = row_data["N"] if "N" in row_data else ""
    attrs["employee_name"]       = row_data["M"] if "M" in row_data else ""
    attrs["progress"]            = row_data["L"].replace("%", "") if "L" in row_data else ""
    attrs["excel_task_status"]   = EXCEL_TASK_STATUS[row_data["P"] if "P" in row_data else ""]
    attrs["phase_name"]          = EXCEL_TYPE_PARENT_TASK[row_data["H"] if "H" in row_data else ""]
    attrs["expected_start_date"] = parser.parse(row_data["E"]) if "E" in row_data and row_data["E"] != "" else None
    attrs["expected_end_date"]   = parser.parse(row_data["F"]) if "F" in row_data and row_data["F"] != "" else None
    attrs["new_end_date"]        = parser.parse(row_data["G"]) if "G" in row_data and row_data["G"] != "" else None
    attrs["expected_time"]       = float(row_data["I"]) if "I" in row_data and row_data["I"] != "" else 0.0

    # expected_time = float(cell["I"]) if "I" in cell and cell["I"] != "" else 0.0
    attrs["task_status"]         = EXCEL_TASK_STATUS[row_data["P"] if "P" in row_data else ""]
    attrs["task_priority"]       = EXCEL_TASK_PRIORITY[row_data["K"] if "K" in row_data else ""]

    # self.task_number = num
    attrs["subject"]             = row_data["O"] if "O" in row_data else ""
    # self.project = cell["C"]
    # self.status = task_status
    # self.priority = task_priority
    # attrs["progress"]            = cell["L"].replace("%", "")
    # self.expected_time = expected_time
    # self.employee_name = cell["M"]
    return attrs

In [12]:
row_date_header = await get_row_date_header()

In [13]:
all_records = await ggSheet.get_all_row_values()

In [14]:
update_records = {}

In [15]:
def is_empty_row(row_attrs):
    if row_attrs['project_code'] == "":
        return True

In [16]:
def handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company):
    print(f"  Handle parent task at row: {idx}\n"
          f"  import_key: {row_attrs['import_key']}\n"
          f"  project_code: {row_attrs['project_code']}\n"
          f"  task: {row_attrs['task']}")

In [24]:
def handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, parent_task, company):
    time_logs_date, time_logsdate_string = mapping_cell_with_dates_raw(row_data, row_date_header)
    print(f"  Handle task at row: {idx}\n"
          f"  import_key: {row_attrs['import_key']}\n"
          f"  project_code: {row_attrs['project_code']}\n"
          f"  employee_name: {row_attrs['employee_name']}\n"
          f"  task: {row_attrs['task']}\n"
          f"  parent task: {parent_task}\n"
          f"  time_logs: {time_logs_date}")

In [18]:
current_parent_task = 0

In [19]:
# if row is empty
    # skip this row
# if col_B == "P"
    # handle as parent task
    # if col_A is not empty (parent task is created)
        # handle update parent task if any changes (compare hash)
    # else
        # handle as new parent task
# else
    # handle as task with timesheet
    # if col_A is not empty (timesheet is created)
        # handle update timesheet if any changes (compare hash)
    # else
        # handle as new timesheet
def handle_timesheet_row(idx, row_data, row_date_header, company):
    global current_parent_task
    row_attrs = mapping_cell_with_attr(row_data)
    if is_empty_row(row_attrs):
        return
    
    if row_attrs['type'] == "P":
        # handle as parent task
        handle_timesheet_row_with_parent_task(idx, row_data, row_attrs, row_date_header, company)
        current_parent_task = idx
    else:
        # handle as task with timesheet
        handle_timesheet_row_as_task(idx, row_data, row_attrs, row_date_header, current_parent_task, company)

In [20]:
async def handle_timesheet_sheet():
    try:
        row_date_header = await get_row_date_header()
        
        current_parent_task = 0
        for idx, row_data in enumerate(all_records):
            if idx >= row_start and idx <= row_end:
                print(f"idx: {idx}")
                # print(row_data)
                handle_timesheet_row(idx, row_data, row_date_header, company)
                # break
    except Exception as err:
        print(f"Handle timesheet {worksheet_name} failed with: {err}")

In [28]:
await handle_timesheet_sheet()

idx: 5
  Handle parent task at row: 5
  import_key: 
  project_code: HR01_ACO.00
  task: 
idx: 6
  Handle task at row: 6
  import_key: 6001 Riza Ave
  project_code: HR01_ACO.00
  task: Nghỉ lễ
  parent task: 5
  time_logs: {datetime.datetime(2024, 4, 27, 0, 0): '8', datetime.datetime(2024, 4, 29, 0, 0): '8', datetime.datetime(2024, 4, 30, 0, 0): '8', datetime.datetime(2024, 5, 1, 0, 0): '8'}
idx: 7
  Handle task at row: 7
  import_key: 
  project_code: HR01_ACO.00
  task: Nghỉ lễ
  parent task: 5
  time_logs: {datetime.datetime(2024, 4, 27, 0, 0): '8', datetime.datetime(2024, 4, 29, 0, 0): '8', datetime.datetime(2024, 4, 30, 0, 0): '8', datetime.datetime(2024, 5, 1, 0, 0): '8'}
idx: 8
  Handle task at row: 8
  import_key: 
  project_code: HR01_ACO.00
  task: Nghỉ lễ
  parent task: 5
  time_logs: {datetime.datetime(2024, 4, 27, 0, 0): '8', datetime.datetime(2024, 4, 29, 0, 0): '8', datetime.datetime(2024, 4, 30, 0, 0): '8', datetime.datetime(2024, 5, 1, 0, 0): '8'}
idx: 9
  Handle task 

In [33]:
# url_file = "https://docs.google.com/spreadsheets/d/1rgen0jYUoAzHRelVi6c2ATJs3IzYPD20wMcT63inKoI/edit#gid=996503932"
# worksheet_name="T2"

In [34]:
# ggSheet = GGSheet(url_file, worksheet_name)

In [ ]:
# ggSheet.open_worksheet(url_file)

In [35]:
# all_records = await ggSheet.get_all_row_values()

In [25]:
await handle_timesheet_sheet()

idx: 5
  Handle parent task at row: 5
  import_key: 7590380--dm4ammu9aa
  project_code: P33422_HTP.00
  task: THIẾT KẾ THI CÔNG
idx: 6
  Handle task at row: 6
  import_key: 74930219--m3u48b0dm9--TS-22-04-24-01078_HR-EMP-00020
  project_code: P33422_HTP.00
  employee_name: Bình Trịnh Thanh
  task: Thuyết minh TKTC
  parent task: 5
  time_logs: {}
idx: 7
  Handle task at row: 7
  import_key: 67647752--m3uf45mtnt--TS-22-04-24-01079_HR-EMP-00069
  project_code: P33422_HTP.00
  employee_name: Huy Phan Trịnh Hoàng
  task: Thuyết minh TKTC
  parent task: 5
  time_logs: {}
idx: 8
  Handle task at row: 8
  import_key: 12054813--m3urvr48hh--TS-22-04-24-01080_HR-EMP-00020
  project_code: P33422_HTP.00
  employee_name: Bình Trịnh Thanh
  task: Chép mô hình, bảng tính, ... lên máy chủ
  parent task: 5
  time_logs: {}
idx: 9
  Handle task at row: 9
  import_key: 3816877--m3v6mglnkm--TS-22-04-24-01081_HR-EMP-00050
  project_code: P33422_HTP.00
  employee_name: Sokkheang Chan
  task: Thuyết minh TKTC
